In [1]:
import os
import sys
import matplotlib.pyplot as plt
import cv2
import random
import numpy as np
from tqdm.auto import tqdm

In [2]:
os.getcwd()

'/home/jaeehooon/PycharmProjects/processing_52_2'

In [3]:
data_root = '../dataset/52_2/'
final_root = os.path.join(data_root, 'final')
final_rgb_dir = os.path.join(final_root, 'RGBFolder')
final_label_dir = os.path.join(final_root, 'LabelFolder')
final_modalX_dir = os.path.join(final_root, 'ModalXFolder')

print('RGBFolder: ', os.path.exists(final_rgb_dir))
print('LabelFolder: ', os.path.exists(final_label_dir))
print('ModalXFolder: ', os.path.exists(final_modalX_dir))


RGBFolder:  True
LabelFolder:  True
ModalXFolder:  True


In [4]:
label_fn_list = os.listdir(final_label_dir)
print(len(label_fn_list))

2915


In [5]:
label_fn_list = [os.path.splitext(fn)[0] for fn in label_fn_list]
print(label_fn_list[:2])

['20220906_154332_26', '20220719_180415_31']


In [6]:
def get_target_data(path):
    target_list = []
    for (root, dir, files) in os.walk(path):
        if 'calibration' in root and len(files) > 0 and files[0] != '.DS_Store':
            target_list.append(
                root.split('/')[:-1]
            )
    return target_list

target_data = get_target_data(data_root)

In [7]:
target_data

[['..', 'dataset', '52_2', 'DOGU', 'CP', 'a', 'cloudy', '220719', '17-19'],
 ['..', 'dataset', '52_2', 'RASTECH', 'DCC', 'D', 'cloudy', '220901', '11-14'],
 ['..', 'dataset', '52_2', 'RASTECH', 'DCC', 'D', 'cloudy', '220902', '14-19'],
 ['..', 'dataset', '52_2', 'RASTECH', 'DCC', 'D', 'sunny', '220908', '14-19'],
 ['..', 'dataset', '52_2', 'RASTECH', 'DCC', 'D', 'sunny', '220915', '11-14'],
 ['..', 'dataset', '52_2', 'RASTECH', 'DCC', 'D', 'sunny', '220906', '14-19'],
 ['..', 'dataset', '52_2', 'RASTECH', 'DCC', 'D', 'sunny', '220905', '14-19'],
 ['..', 'dataset', '52_2', 'RASTECH', 'DCC', 'D', 'sunny', '220805', '14-19']]

## train_test example 1

- DOGU만 train, RASTECH는 test
    - Train: 0.73%
    - Test: 0.27%

In [9]:
total_file_path_list = []

train_list, test_list = [], []

for target in target_data:
    files = os.listdir(os.path.join(
        '/'.join(target),
        'seg',
        'segmentation',
    ))
    files = [os.path.splitext(fn)[0] for fn in files if not os.path.isdir(fn)]
    common_files = list(set(label_fn_list) & set(files))

    if 'DOGU' in target:
        train_list.extend(common_files)
    else:
        test_list.extend(common_files)

In [10]:
print(len(train_list))
print(len(test_list))

2134
781


In [11]:
print("Train: {:,.2f}%".format(len(train_list) / len(label_fn_list)))
print("Test: {:,.2f}%".format(len(test_list) / len(label_fn_list)))

Train: 0.73%
Test: 0.27%


In [12]:
label_fn_list[0]

'20220906_154332_26'

In [9]:
class_names = {}

with open('./class_names.txt', 'r') as f:
    lines = f.readlines()
    
    for line in lines:
        cls_name = line.rstrip('\n')
        if cls_name not in class_names:
            class_names[cls_name] = len(class_names)

class_names

{'unlabeled': 0,
 'OUT_Road': 1,
 'OUT_Structure': 2,
 'OUT_Tree': 3,
 'OUT_Fence': 4,
 'OUT_Pole': 5,
 'OUT_Sign': 6,
 'Vehicle': 7,
 'OUT_Curbstone': 8,
 'OUT_Pavement': 9,
 'OUT_Undefined Stuff': 10,
 'IN_Hall Way': 11,
 'IN_Open Space': 12,
 'IN_Wall': 13,
 'IN_Pillar': 14,
 'IN_Obstruction': 15,
 'IN_Gate': 16,
 'IN_Elevator': 17,
 'IN_Sign': 18,
 'IN_Fence': 19,
 'IN_Undefined Stuff': 20,
 'OUT_Grass': 21,
 'Human': 22,
 'OUT_Bench': 23,
 'Stroller': 24,
 'OUT_Sculpture': 25,
 'OUT_Traffic Safety Stuff': 26,
 'Undefined Object': 27,
 'OUT_Gate': 28,
 'Two-wheeled Vehicle': 29,
 'Animal': 30,
 'Wheelchair': 31,
 'IN_Bench': 32,
 'IN_Emergency Stuff': 33}

In [14]:
predefined_class = {
    'unlabeled': 0,
    'OUT_Road': 1,
    'OUT_Structure': 2,
    'OUT_Tree': 3,
    'OUT_Fence': 4,
    'OUT_Pole': 5,
    'OUT_Sign': 6,
    'Vehicle': 7,
    'OUT_Curbstone': 8,
    'OUT_Pavement': 9,
    'OUT_Undefined Stuff': 10,
    'IN_Hall Way': 11,
    'IN_Open Space': 12,
    'IN_Wall': 13,
    'IN_Pillar': 14,
    'IN_Obstruction': 15,
    'IN_Gate': 16,
    'IN_Elevator': 17,
    'IN_Sign': 18,
    'IN_Fence': 19,
    'IN_Undefined Stuff': 20,
    'OUT_Grass': 21,
    'Human': 22,
    'OUT_Bench': 23,
    'Stroller': 24,
    'OUT_Sculpture': 25,
    'OUT_Traffic Safety Stuff': 26,
    'Undefined Object': 27,
    'OUT_Gate': 28,
    'Two-wheeled Vehicle': 29,
    'Animal': 30,
    'Wheelchair': 31,
    'IN_Bench': 32,
    'IN_Emergency Stuff': 33
}

In [15]:
class_names == predefined_class

True

In [11]:
reverse_class_names = {v: k for k, v in class_names.items()}
reverse_class_names

{0: 'unlabeled',
 1: 'OUT_Road',
 2: 'OUT_Structure',
 3: 'OUT_Tree',
 4: 'OUT_Fence',
 5: 'OUT_Pole',
 6: 'OUT_Sign',
 7: 'Vehicle',
 8: 'OUT_Curbstone',
 9: 'OUT_Pavement',
 10: 'OUT_Undefined Stuff',
 11: 'IN_Hall Way',
 12: 'IN_Open Space',
 13: 'IN_Wall',
 14: 'IN_Pillar',
 15: 'IN_Obstruction',
 16: 'IN_Gate',
 17: 'IN_Elevator',
 18: 'IN_Sign',
 19: 'IN_Fence',
 20: 'IN_Undefined Stuff',
 21: 'OUT_Grass',
 22: 'Human',
 23: 'OUT_Bench',
 24: 'Stroller',
 25: 'OUT_Sculpture',
 26: 'OUT_Traffic Safety Stuff',
 27: 'Undefined Object',
 28: 'OUT_Gate',
 29: 'Two-wheeled Vehicle',
 30: 'Animal',
 31: 'Wheelchair',
 32: 'IN_Bench',
 33: 'IN_Emergency Stuff'}

In [17]:
class_count = dict({k: 0 for k in class_names.keys()})    # instance 개수 고려 하지 않음. 즉, Max: train 파일 개수, Min: 0

for label_fn in tqdm(label_fn_list):
    label_img = cv2.imread(os.path.join(final_label_dir, "{}.png".format(label_fn)), cv2.IMREAD_GRAYSCALE)
    for val in np.unique(label_img):
        class_count[reverse_class_names[val]] += 1

  0%|          | 0/2915 [00:00<?, ?it/s]

In [18]:
class_count

{'unlabeled': 2636,
 'OUT_Road': 1614,
 'OUT_Structure': 1027,
 'OUT_Tree': 2094,
 'OUT_Fence': 976,
 'OUT_Pole': 840,
 'OUT_Sign': 507,
 'Vehicle': 45,
 'OUT_Curbstone': 1384,
 'OUT_Pavement': 1681,
 'OUT_Undefined Stuff': 1049,
 'IN_Hall Way': 75,
 'IN_Open Space': 800,
 'IN_Wall': 815,
 'IN_Pillar': 57,
 'IN_Obstruction': 705,
 'IN_Gate': 740,
 'IN_Elevator': 78,
 'IN_Sign': 100,
 'IN_Fence': 133,
 'IN_Undefined Stuff': 78,
 'OUT_Grass': 1326,
 'Human': 1505,
 'OUT_Bench': 737,
 'Stroller': 49,
 'OUT_Sculpture': 342,
 'OUT_Traffic Safety Stuff': 5,
 'Undefined Object': 108,
 'OUT_Gate': 81,
 'Two-wheeled Vehicle': 4,
 'Animal': 10,
 'Wheelchair': 12,
 'IN_Bench': 88,
 'IN_Emergency Stuff': 261}

In [19]:
if not os.path.exists('./train.txt'):
    with open('./train.txt', 'w') as f:
        f.write('\n'.join(train_list)) 
else:
    print("Already exists!")

Already exists!


In [20]:
if not os.path.exists('./test.txt'):
    with open('./test.txt', 'w') as f:
        f.write('\n'.join(test_list))
else:
    print("Already exists!")

Already exists!


--- 

- train/test 내의 class 비율
    - 마찬가지로 instance 비율은 따지지 않음
    
    - 해당 클래스가 있는지 없는지 여부만 확인

In [23]:
class_count = dict({k: 0 for k in class_names.keys()})    # instance 개수 고려 하지 않음. 즉, Max: train 파일 개수, Min: 0

with open('./train.txt', 'r') as f:
    train_fn_list = f.readlines()

train_fn_list = [fn.rstrip('\n') for fn in train_fn_list]
for label_fn in tqdm(train_fn_list):
    label_img = cv2.imread(os.path.join(final_label_dir, "{}.png".format(label_fn)), cv2.IMREAD_GRAYSCALE)
    for val in np.unique(label_img):
        class_count[reverse_class_names[val]] += 1
        
class_count

  0%|          | 0/2134 [00:00<?, ?it/s]

{'unlabeled': 1889,
 'OUT_Road': 1606,
 'OUT_Structure': 993,
 'OUT_Tree': 2065,
 'OUT_Fence': 976,
 'OUT_Pole': 840,
 'OUT_Sign': 500,
 'Vehicle': 45,
 'OUT_Curbstone': 1368,
 'OUT_Pavement': 1646,
 'OUT_Undefined Stuff': 1021,
 'IN_Hall Way': 62,
 'IN_Open Space': 67,
 'IN_Wall': 67,
 'IN_Pillar': 21,
 'IN_Obstruction': 10,
 'IN_Gate': 18,
 'IN_Elevator': 58,
 'IN_Sign': 25,
 'IN_Fence': 62,
 'IN_Undefined Stuff': 20,
 'OUT_Grass': 1326,
 'Human': 1373,
 'OUT_Bench': 735,
 'Stroller': 49,
 'OUT_Sculpture': 342,
 'OUT_Traffic Safety Stuff': 3,
 'Undefined Object': 66,
 'OUT_Gate': 71,
 'Two-wheeled Vehicle': 4,
 'Animal': 10,
 'Wheelchair': 12,
 'IN_Bench': 0,
 'IN_Emergency Stuff': 0}

In [24]:
class_count = dict({k: 0 for k in class_names.keys()})    # instance 개수 고려 하지 않음. 즉, Max: train 파일 개수, Min: 0

with open('./test.txt', 'r') as f:
    train_fn_list = f.readlines()

train_fn_list = [fn.rstrip('\n') for fn in train_fn_list]
for label_fn in tqdm(train_fn_list):
    label_img = cv2.imread(os.path.join(final_label_dir, "{}.png".format(label_fn)), cv2.IMREAD_GRAYSCALE)
    for val in np.unique(label_img):
        class_count[reverse_class_names[val]] += 1
        
class_count

  0%|          | 0/781 [00:00<?, ?it/s]

{'unlabeled': 747,
 'OUT_Road': 8,
 'OUT_Structure': 34,
 'OUT_Tree': 29,
 'OUT_Fence': 0,
 'OUT_Pole': 0,
 'OUT_Sign': 7,
 'Vehicle': 0,
 'OUT_Curbstone': 16,
 'OUT_Pavement': 35,
 'OUT_Undefined Stuff': 28,
 'IN_Hall Way': 13,
 'IN_Open Space': 733,
 'IN_Wall': 748,
 'IN_Pillar': 36,
 'IN_Obstruction': 695,
 'IN_Gate': 722,
 'IN_Elevator': 20,
 'IN_Sign': 75,
 'IN_Fence': 71,
 'IN_Undefined Stuff': 58,
 'OUT_Grass': 0,
 'Human': 132,
 'OUT_Bench': 2,
 'Stroller': 0,
 'OUT_Sculpture': 0,
 'OUT_Traffic Safety Stuff': 2,
 'Undefined Object': 42,
 'OUT_Gate': 10,
 'Two-wheeled Vehicle': 0,
 'Animal': 0,
 'Wheelchair': 0,
 'IN_Bench': 88,
 'IN_Emergency Stuff': 261}

In [44]:
# y = list(class_count.values())
# x = np.arange(len(class_count))
# xlabel = list(class_count.keys())

# plt.figure(figsize=(20, 15))
# plt.title("# of class per Image")
# plt.bar(x, y)
# plt.xticks(x, xlabel)
# plt.yticks(sorted(y))
# plt.xlabel("Class Names")
# plt.ylabel("Count")
# plt.show()

In [45]:
# labels = list(class_count.keys())
# sizes = list(class_count.values())
# # colors = ['yellowgreen', 'gold', 'lightskyblue', 'lightcoral']
# plt.figure(figsize=(20, 15))
# explode = tuple([0] * len(class_count))
# plt.title("Class Count")
# plt.pie(sizes, explode=explode, labels=labels, colors=colors,
#         autopct='%1.1f%%', shadow=True, startangle=90, labeldistance=1.1)
# plt.axis('equal')
# plt.show()

## train_test example 2

- 전체 데이터 중, train:test = 7:3 
- random split

In [8]:
total_file_path_list = []

train_ratio = 0.7
train_list = random.sample(label_fn_list, int(len(label_fn_list) * train_ratio))
test_list = list(set(label_fn_list) - set(train_list))

In [51]:
print(len(train_list))
print(len(test_list))

2040
875


In [52]:
print("Train: {:,.2f}%".format(len(train_list) / len(label_fn_list)))
print("Test: {:,.2f}%".format(len(test_list) / len(label_fn_list)))

Train: 0.70%
Test: 0.30%


In [53]:
if not os.path.exists('./train_random_split.txt'):
    with open('./train_random_split.txt', 'w') as f:
        f.write('\n'.join(train_list)) 
else:
    print("Already exists!")

In [54]:
if not os.path.exists('./test_random_split.txt'):
    with open('./test_random_split.txt', 'w') as f:
        f.write('\n'.join(test_list))
else:
    print("Already exists!")

In [13]:
class_count = dict({k: 0 for k in class_names.keys()})    # instance 개수 고려 하지 않음. 즉, Max: train 파일 개수, Min: 0

with open('./train_random_split.txt', 'r') as f:
    train_fn_list = f.readlines()

train_fn_list = [fn.rstrip('\n') for fn in train_fn_list]
for label_fn in tqdm(train_fn_list):
    label_img = cv2.imread(os.path.join(final_label_dir, "{}.png".format(label_fn)), cv2.IMREAD_GRAYSCALE)
    for val in np.unique(label_img):
        class_count[reverse_class_names[val]] += 1
        
class_count

  0%|          | 0/2040 [00:00<?, ?it/s]

{'unlabeled': 1846,
 'OUT_Road': 1127,
 'OUT_Structure': 701,
 'OUT_Tree': 1461,
 'OUT_Fence': 670,
 'OUT_Pole': 595,
 'OUT_Sign': 347,
 'Vehicle': 25,
 'OUT_Curbstone': 976,
 'OUT_Pavement': 1178,
 'OUT_Undefined Stuff': 726,
 'IN_Hall Way': 48,
 'IN_Open Space': 564,
 'IN_Wall': 573,
 'IN_Pillar': 45,
 'IN_Obstruction': 501,
 'IN_Gate': 526,
 'IN_Elevator': 48,
 'IN_Sign': 67,
 'IN_Fence': 91,
 'IN_Undefined Stuff': 55,
 'OUT_Grass': 930,
 'Human': 1061,
 'OUT_Bench': 531,
 'Stroller': 35,
 'OUT_Sculpture': 229,
 'OUT_Traffic Safety Stuff': 2,
 'Undefined Object': 83,
 'OUT_Gate': 57,
 'Two-wheeled Vehicle': 3,
 'Animal': 10,
 'Wheelchair': 11,
 'IN_Bench': 61,
 'IN_Emergency Stuff': 182}

In [12]:
class_count = dict({k: 0 for k in class_names.keys()})    # instance 개수 고려 하지 않음. 즉, Max: train 파일 개수, Min: 0

with open('./test_random_split.txt', 'r') as f:
    train_fn_list = f.readlines()

train_fn_list = [fn.rstrip('\n') for fn in train_fn_list]
for label_fn in tqdm(train_fn_list):
    label_img = cv2.imread(os.path.join(final_label_dir, "{}.png".format(label_fn)), cv2.IMREAD_GRAYSCALE)
    for val in np.unique(label_img):
        class_count[reverse_class_names[val]] += 1
        
class_count

  0%|          | 0/875 [00:00<?, ?it/s]

{'unlabeled': 790,
 'OUT_Road': 487,
 'OUT_Structure': 326,
 'OUT_Tree': 633,
 'OUT_Fence': 306,
 'OUT_Pole': 245,
 'OUT_Sign': 160,
 'Vehicle': 20,
 'OUT_Curbstone': 408,
 'OUT_Pavement': 503,
 'OUT_Undefined Stuff': 323,
 'IN_Hall Way': 27,
 'IN_Open Space': 236,
 'IN_Wall': 242,
 'IN_Pillar': 12,
 'IN_Obstruction': 204,
 'IN_Gate': 214,
 'IN_Elevator': 30,
 'IN_Sign': 33,
 'IN_Fence': 42,
 'IN_Undefined Stuff': 23,
 'OUT_Grass': 396,
 'Human': 444,
 'OUT_Bench': 206,
 'Stroller': 14,
 'OUT_Sculpture': 113,
 'OUT_Traffic Safety Stuff': 3,
 'Undefined Object': 25,
 'OUT_Gate': 24,
 'Two-wheeled Vehicle': 1,
 'Animal': 0,
 'Wheelchair': 1,
 'IN_Bench': 27,
 'IN_Emergency Stuff': 79}